## Toronto Neighborhood Clustering Assignment

In [1]:
%matplotlib inline 
%pylab inline
from IPython.display import display

# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
#import geocode

import matplotlib.pyplot as plt
from matplotlib import colors

#!conda install -c conda-forge folium=0.5.0 
import folium
import requests

from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Populating the interactive namespace from numpy and matplotlib


## Pandas method

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(url)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df = df.drop(df[df['Borough']== 'Not assigned'].index)
df.reset_index(drop = True, inplace = True)

# Check if a missing neighbourhood has a Borough, and assign Borough to such cases 
if df[df.Neighbourhood == 'Not assigned'].shape[0] != 0:
    df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.loc[df.Neighbourhood == 'Not assigned', 'Borough']
    
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

## Alternative Beautiful Soup Scraper

In [5]:
def getTableFromWiki(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    all_tables=soup.find_all("table")
    right_table=soup.find('table', class_='wikitable sortable')
    
    A=[]
    B=[]
    C=[]

    for row in right_table.findAll('tr'):
        cells=row.findAll('td')
        if len(cells)==3:
            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            C.append(cells[2].find(text=True))
    
    df = pd.DataFrame(A, columns = ['Postal Code'])
    df['Borough'] = B
    df['Neighbourhood'] = C
    
    return df

In [6]:
df_test = getTableFromWiki(url)

df_test = df_test.replace('\n','', regex=True)
df_test = df_test.drop(df[df['Borough']== 'Not assigned'].index)

## Geocode

df['Latitude'] = np.nan

df['Longitude'] = np.nan

for i in range(len(df)):

    postal = df.loc[i, 'Postal Code']
    g = geocoder.google('{}, Toronto, Ontario'.format(postal))
    lat_lng_coords = g.latlng
    
    df.loc[i, 'Latitude'] = lat_lng_coords[0]
    df.loc[i, 'Longitude'] = lat_lng_coords[1]
    


## CSV method

In [7]:
coor = pd.read_csv('http://cocl.us/Geospatial_data')

In [8]:
df = pd.merge(df, coor, on = ['Postal Code'])

In [9]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Clustering

In [10]:
np.random.seed(0)

In [11]:
# create map of Toronto using latitude and longitude values
latitude = 42.034534
longitude = -79.347015

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

display(map_toronto)


In [12]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


To avoid having more of the same neighbourhood, I remove duplicated neighbourhoods 

In [13]:
# Examples
nbh_dupl = df.loc[df['Neighbourhood'].duplicated()]['Neighbourhood'].unique()
df[df['Neighbourhood'].isin(nbh_dupl)]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
7,M3B,North York,Don Mills,43.745906,-79.352188
13,M3C,North York,Don Mills,43.725900,-79.340923
40,M3K,North York,Downsview,43.737473,-79.464763
46,M3L,North York,Downsview,43.739015,-79.506944
53,M3M,North York,Downsview,43.728496,-79.495697
60,M3N,North York,Downsview,43.761631,-79.520999


In [14]:
# Remove duplicates
df = df.loc[~df['Neighbourhood'].duplicated()]

## Foursquare Credentials and Version 

In [15]:
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
VERSION = '***' # Foursquare API version

LIMIT = 100
radius = 500

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        if 'groups' in requests.get(url).json()['response'].keys():
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
        else:
            continue
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [ ]:
toronto_venues.head()

## Analyze each Neighbourhood 

The analysis follows that of the New York analysis

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000

### Write function to sort venues in descending order

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Clothing Store,Lounge,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Athletics & Sports,Coffee Shop,Colombian Restaurant,Dance Studio,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Frozen Yogurt Shop,Ice Cream Shop,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Shopping Mall,Mobile Phone Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Department Store,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Thai Restaurant,Juice Bar,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Indian Restaurant


# Clustering

In [89]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted, on = 'Neighborhood')

In [103]:
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,French Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Health Food Store,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Women's Store,Boutique,Miscellaneous Shop,Furniture / Home Store,Arts & Crafts Store,Event Space,Accessories Store,Vietnamese Restaurant,Coffee Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Gym,Sandwich Place,Restaurant,Park,Mexican Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center


In [106]:
# create map (Toronto coordinates)
latitude = 43.651070
longitude = -79.347015

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
display(map_clusters)

# Examine Clusters

## Cluster 1

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,French Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
7,East York,0,Pizza Place,Pet Store,Breakfast Spot,Gym / Fitness Center,Pharmacy,Bank,Athletics & Sports,Gastropub,Café,Intersection
9,North York,0,Japanese Restaurant,Pizza Place,Bakery,Pub,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
14,Etobicoke,0,Pet Store,Convenience Store,Beer Store,Shopping Plaza,Liquor Store,Café,Coffee Shop,Pizza Place,Pharmacy,Colombian Restaurant
45,North York,0,Gym,Pizza Place,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant
46,Scarborough,0,American Restaurant,Motel,Skating Rink,Women's Store,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
63,Etobicoke,0,Pizza Place,Discount Store,Coffee Shop,Intersection,Sandwich Place,Middle Eastern Restaurant,Chinese Restaurant,Women's Store,Department Store,Dessert Shop
65,North York,0,Grocery Store,Pizza Place,Bank,Coffee Shop,Butcher,Pharmacy,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Deli / Bodega
75,Scarborough,0,Pizza Place,Intersection,Bank,Pharmacy,Italian Restaurant,Noodle House,Gas Station,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant
82,Etobicoke,0,Grocery Store,Pizza Place,Liquor Store,Beer Store,Fried Chicken Joint,Sandwich Place,Discount Store,Fast Food Restaurant,Pharmacy,Drugstore


Cluster 1 appear to be 

## Cluster 2

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Health Food Store,Beer Store,Shoe Store
3,North York,1,Clothing Store,Women's Store,Boutique,Miscellaneous Shop,Furniture / Home Store,Arts & Crafts Store,Event Space,Accessories Store,Vietnamese Restaurant,Coffee Shop
4,Downtown Toronto,1,Coffee Shop,Diner,Gym,Sandwich Place,Restaurant,Park,Mexican Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
6,North York,1,Gym,Caribbean Restaurant,Japanese Restaurant,Café,Athletics & Sports,Concert Hall,Construction & Landscaping,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,Downtown Toronto,1,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Café,Fast Food Restaurant,Tea Room,Bookstore
10,Scarborough,1,Bar,Home Service,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Deli / Bodega
11,East York,1,Curling Ice,Park,Pharmacy,Beer Store,Skating Rink,Athletics & Sports,Dance Studio,Distribution Center,Department Store,Dessert Shop
12,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Gym,Breakfast Spot,Moroccan Restaurant
13,York,1,Field,Tennis Court,Hockey Arena,Trail,Women's Store,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
15,Scarborough,1,Restaurant,Medical Center,Rental Car Location,Mexican Restaurant,Intersection,Bank,Electronics Store,Breakfast Spot,Doner Restaurant,Dog Run


Cluster 2 appear to be downtown neighborhoods, with a lot of coffe shops, parks, stores and bars 

## Cluster 3

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant


Cluster 3 is a single neighborhood in the outskirts of Toronto 

## Cluster 4

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Park,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant
18,York,3,Park,Women's Store,Pool,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant
37,North York,3,Park,Airport,Electronics Store,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center
54,Central Toronto,3,Park,Swim School,Bus Line,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Event Space,Distribution Center
61,Central Toronto,3,Park,Sushi Restaurant,Jewelry Store,Trail,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
70,Etobicoke,3,Park,Sandwich Place,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run
78,Scarborough,3,Park,Playground,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant
84,Downtown Toronto,3,Park,Playground,Trail,Distribution Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
90,Etobicoke,3,Park,Pool,River,College Rec Center,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
93,Etobicoke,3,Park,Deli / Bodega,Baseball Field,Women's Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run


Cluster 4 appear to be park neighborhoods

## Cluster 5 

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,East York,4,Park,Convenience Store,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
47,North York,4,Park,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
57,York,4,Convenience Store,Park,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
59,North York,4,Park,Convenience Store,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Cluster 5 appear to be store and park neighborhoods 